In [1]:
from federated_learning.utils import SHAPUtil
from federated_learning import ClientPlane, Configuration
from federated_learning.server import Server
from federated_learning.observer import ServerObserver

In [2]:
class ObserverConfiguration():
    experiment_type = "shap_fl_clean"
    experiment_id = 0
    test = True
    dataset_type = "MNIST"
    
    # Client Configurations 
    client_name = "client"
    client_type = "client"
    
    # Server Configurations 
    server_name = "server"
    server_type = "server"
    server_id = 0

In [3]:
config = Configuration()
data = config.DATASET(config)
shap_util = SHAPUtil(data.test_dataloader)
observer_config = ObserverConfiguration()

MNIST training data loaded.
MNIST test data loaded.


In [4]:
server = Server(config, data.test_dataloader, shap_util)
server.observer = ServerObserver(config, observer_config)
client_plane = ClientPlane(config, observer_config, data, shap_util)

def set_rounds(rounds):
    client_plane.set_rounds(rounds)
    server.set_rounds(rounds)
    
def run_round(rounds):
    client_plane.update_clients(server.get_nn_parameters())
    selected_clients = server.select_clients()
    client_parameters = client_plane.train_selected_clients(selected_clients)
    server.aggregate_model(client_parameters)
    server.test()
    server.push_metrics()
    
for i in range(200):
    set_rounds(i)
    run_round(i)
    print("Round {} finished".format(i+1))

Load model successfully
Create 200 clients with dataset of size 300

Server Test set: Average loss: 0.0023, Accuracy: 1748/10000 (17%)



Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.


Predictions: tensor([[0],
        [0],
        [7],
        [7],
        [0],
        [0],
        [0],
        [7],
        [0],
        [7]])
Successfully pushed client data to victoria metrics
Round 1 finished

Server Test set: Average loss: 0.0023, Accuracy: 2003/10000 (20%)

Predictions: tensor([[0],
        [0],
        [0],
        [7],
        [9],
        [0],
        [0],
        [7],
        [0],
        [0]])
Successfully pushed client data to victoria metrics
Round 2 finished

Server Test set: Average loss: 0.0022, Accuracy: 3212/10000 (32%)

Predictions: tensor([[0],
        [7],
        [2],
        [7],
        [9],
        [0],
        [0],
        [7],
        [0],
        [0]])
Successfully pushed client data to victoria metrics
Round 3 finished

Server Test set: Average loss: 0.0020, Accuracy: 5374/10000 (54%)

Predictions: tensor([[0],
        [1],
        [2],
        [1],
        [9],
        [0],
        [0],
        [7],
        [0],
        [0]])
Successfully 

In [ ]:
server.get_shap_values()